In [ ]:
# example_meta_data_reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [ ]:

import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib widget
# %matplotlib qt

In [ ]:
# Example loading a directory (highly recommend using lazy loading)
example_set = ak.from_parquet('/home/unimelb.edu.au/smeighenberg/data/triangle_pone_one/', lazy=True)

In [ ]:
# Example loading a single file
# example_set = ak.from_parquet('./output/custom_1337_meta_data.parquet')

In [ ]:
# The events are split into three parts:
#   - lepton: the light from the generated lepton
#   - hadron: the light produced from the associated hadronic cascade (at production point)
#   - total: the light from both parts
# Each of these fields is formatted the same way
example_set.fields

In [ ]:
# Example for one event
event_id = 0
# The sensors hit
print("Total number of hits:")
print(len(example_set.total.sensor_id[event_id]))
print('The hit sensors: ')
print(example_set.total.sensor_id[event_id])
print('Their associated strings')
print(example_set.total.sensor_string_id[event_id])
print('The hit times')
print(example_set.total.t[event_id])

In [ ]:
def parquet_sub_plotting(
        data, e_id, det,
        channel='total',
        show_lepton=True,
        scaling=20,
        number_of_hits=1000,
        hitcut=3
    ):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    hit_counts = []
    subset_ids = np.random.randint(
        0,
        len(data[channel].sensor_pos_x[e_id])-1,
        size=number_of_hits
    )
    print('Total amount of hits: %d' %len(data[channel].sensor_pos_x[e_id]))
    print('Using a subset of %d for plotting' %number_of_hits)
    event_id = e_id
    sensor_comb = np.array([
        [data[channel].sensor_pos_x[event_id][subset_ids][i],
        data[channel].sensor_pos_y[event_id][subset_ids][i],
        data[channel].sensor_pos_z[event_id][subset_ids][i]
        ]
        for i in range(number_of_hits)
    ])
    print('Fetching unique modules and their hits')
    hit_doms, hit_counts = np.unique(sensor_comb, axis=0, return_counts=True)
    hit_counts = np.log10(hit_counts)
    hit_counts[hit_counts < np.log10(hitcut)] = 0.
    print('Finished the modules')
    print('Plotting...')
    ax.scatter(
        hit_doms[:, 0],
        hit_doms[:, 1],
        hit_doms[:, 2],
        s=(
            hit_counts * scaling
        ),
        color='r', alpha=0.5)
    print('Finished modules')
    if show_lepton:
        print('Plotting lepton...')
        ax.scatter(
            data.primary_lepton_1.loss_pos_x[event_id],
            data.primary_lepton_1.loss_pos_y[event_id],
            data.primary_lepton_1.loss_pos_z[event_id],
            s=( scaling
            ),
            color='blue', alpha=0.5)
        print('Finished lepton')
    ax.scatter(
        det[:, 0],
        det[:, 1],
        det[:, 2],
        s=1,
        color='k', alpha=0.5)
    ax.set_xlim(min(det[:, 0]), max(det[:, 0]))
    ax.set_ylim(min(det[:, 1]), max(det[:, 1]))
    ax.set_zlim(min(det[:, 2]), max(det[:, 2]))
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('Z [m]')
    plt.title(
        'Event %d with pdg id = %d and energy %.1f GeV' % (
            event_id,
            data.mc_truth.injection_type[event_id],
            data.mc_truth.injection_energy[event_id]
            )
    )
    plt.show()

In [ ]:
# Getting the detector
pos = []
with open('/home/unimelb.edu.au/smeighenberg/Projects/hebe/hebe/data/pone_triangle-geo') as geo_in:
    read_lines = geo_in.readlines()
    modules_i = read_lines.index("### Modules ###\n")   
    for line in read_lines[modules_i+1:]:
        line = line.strip("\n").split("\t")
        pos.append(
            np.array([float(line[0]), float(line[1]),
            float(line[2])]))
    det = np.array(pos)
det = det - np.mean(det, axis=0)

In [ ]:
parquet_sub_plotting(
    example_set,
    e_id=12, det=det,
    channel='total',
    show_lepton=True,
    scaling=10,
    number_of_hits=100,
    hitcut=0
)